# Edgar: Comparative Q&A

In [1]:
import os
import sys
from pprint import pprint
import nest_asyncio
nest_asyncio.apply()
from typing import List, Any
from tqdm.autonotebook import trange
from pydantic import BaseModel, Field
import logging


current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

# Langchain imports
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import (
    PromptTemplate,
    load_prompt
)
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.output_parsers import PydanticOutputParser
from langchain.retrievers.multi_query import MultiQueryRetriever

# Llama index imports
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.llms import (
    CustomLLM,
    CompletionResponse,
    CompletionResponseGen,
    LLMMetadata,
)
from llama_index.llms.base import llm_completion_callback


#from utils.sambanova_endpoint import SambaNovaEndpoint
from langchain_community.llms.sambanova import SambaStudio, Sambaverse


from dotenv import load_dotenv
load_dotenv(os.path.join(repo_dir,'.env'))

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

/var/folders/5p/79s0z9ss0dd6__6jc245j5t40000gp/T/ipykernel_33704/3150193592.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


## Llama index

### Uber vs Lift 2021

In [2]:
class SambaNovaLLMWrapper(CustomLLM):
    context_window: int = 3900
    num_output: int = 256
    model_name: str = "sambanova_llama7b"
    
    def _get_sambanova_llm(self):

        #llm = Sambaverse(
        #    sambaverse_model_name='Meta/llama-2-7b-chat-hf',
        #    model_kwargs={
        #        'do_sample': False,
        #        'max_tokens_to_generate': 512,
        #        'select_expert': 'llama-2-7b-chat-hf',
        #        'process_prompt': False,
        #        'temperature': 0.01,
        #    },
        #)

        llm = SambaStudio(
            streaming=True,
            model_kwargs={
                'max_tokens_to_generate': 512,
                'select_expert': 'Meta-Llama-3-70B-Instruct',
                'process_prompt': False,
            },
        )

        return llm

    @property
    def metadata(self) -> LLMMetadata:
        """Get LLM metadata."""
        return LLMMetadata(
            context_window=self.context_window,
            num_output=self.num_output,
            model_name=self.model_name,
        )

    @llm_completion_callback()
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        llm = self._get_sambanova_llm()
        response = llm(prompt)
        return CompletionResponse(text=response)

    @llm_completion_callback()
    def stream_complete(
        self, prompt: str, **kwargs: Any
    ) -> CompletionResponseGen:
        llm = self._get_sambanova_llm()
        llm_response = llm(prompt)
        for token in llm_response:
            response += token
            yield CompletionResponse(text=response, delta=token)

In [3]:
# Instantiate LLM 
llm = SambaNovaLLMWrapper()

In [4]:
# Instatiate embedding model
embedding_model = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [5]:
# Declare service context
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embedding_model)

In [6]:
## Load data 
lyft_docs = SimpleDirectoryReader(
    input_files=[os.path.join(kit_dir,"data/sec-edgar-filings/pdfs/lyft_10k_2021_5pages.pdf")]
).load_data()
uber_docs = SimpleDirectoryReader(
    input_files=[os.path.join(kit_dir,"data/sec-edgar-filings/pdfs/uber_10k_2021_5pages.pdf")]
).load_data()

## Build indices
lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True, service_context=service_context)

uber_index = VectorStoreIndex.from_documents(uber_docs, show_progress=True, service_context=service_context)

## Build query engines
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)

uber_engine = uber_index.as_query_engine(similarity_top_k=3)

Generating embeddings: 100%|██████████| 9/9 [00:17<00:00,  1.93s/it]


In [7]:
# Instantiate query engine tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021"
            ),
        ),
    ),
]

# Instantiate Sub query engine
s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    service_context=service_context
)

In [8]:
## Run queries
response = s_engine.query(
    "Compare and contrast the customer segments and geographies that grew the fastest"
)

print(response)

response = s_engine.query(
    "Compare revenue growth of Uber and Lyft from 2020 to 2021"
)

print(response)

/Users/petrojm/Documents/projects/ASK/fastAPI/ai-starter-kit/edgar_qna/edgar_fastapi/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Generated 4 sub questions.
[uber_10k] Q: What are the customer segments that grew the fastest for Uber
[uber_10k] Q: What are the geographies that grew the fastest for Uber
[lyft_10k] Q: What are the customer segments that grew the fastest for Lyft
[lyft_10k] Q: What are the geographies that grew the fastest for Lyft
[lyft_10k] A:  Unable to determine from the provided context. The provided context appears to be a scanned copy of a 10-K filing for Lyft, but it does not contain readable text. The text is garbled and appears to be a result of an OCR (Optical Character Recognition) error. Therefore, it is not possible to determine the customer segments that grew the fastest for Lyft from the provided context.
[uber_10k] A:  The query is asking about customer segments that grew the fastest for Uber. However, the provided context information does not contain any specific data or information about customer segments or their growth rates. The context appears to be related to Uber's mission, v

## Langchain

### Uber vs Lift 2021

In [9]:
chunk_size = 1000
chunk_overlap = 0

In [10]:
# Load uber data
loader = PyPDFLoader(os.path.join(kit_dir,"data/sec-edgar-filings/pdfs/uber_10k_2021_5pages.pdf"))
data = loader.load()
for document in data:
    document.metadata['company'] = 'Uber'
    document.metadata['year'] = 2021

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
uber_splits = text_splitter.split_documents(data)

In [11]:
# Load lyft data
loader = PyPDFLoader(os.path.join(kit_dir,"data/sec-edgar-filings/pdfs/lyft_10k_2021_5pages.pdf"))
data = loader.load()
for document in data:
    document.metadata['company'] = 'Lyft'
    document.metadata['year'] = 2021

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
lyft_splits = text_splitter.split_documents(data)

In [12]:
splits = [*uber_splits,*lyft_splits]

print(f"{len(uber_splits)} uber split docs")
print(f"{len(lyft_splits)} lyft split docs")
print(f"{len(splits)} all docs")

15 uber split docs
17 lyft split docs
32 all docs


In [13]:
# Load embeddings and create vector store
embedding = HuggingFaceInstructEmbeddings(
    query_instruction="Represent the query for retrieval: "
)

vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

load INSTRUCTOR_Transformer
max_seq_length  512


In [14]:
# create llm object from Sambanova endpoint class
#llm = Sambaverse(
#            sambaverse_model_name='Meta/llama-2-7b-chat-hf',
#            model_kwargs={
#                'do_sample': False,
#                'max_tokens_to_generate': 512,
#                'select_expert': 'llama-2-7b-chat-hf',
#                'process_prompt': False,
#                'temperature': 0.01,
#            },
#        )

llm = SambaStudio(
            streaming=True,
            model_kwargs={
                'max_tokens_to_generate': 512,
                'select_expert': 'Meta-Llama-3-70B-Instruct',
                'process_prompt': False,
            },
        )

In [20]:
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")

class QuestionListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        questions = [question for question in lines if '?' in question]
        return LineList(lines=questions)

output_parser = QuestionListOutputParser()

query_decomposition_prompt = PromptTemplate(
    input_variables=["question"],
    
    template="""[INST] <<SYS>>Decompose a complex query into a list of questions directly and concisely.<<SYS>>
    Query: {question}
    Output: [/INST]""",
)


In [21]:
query_decomposition_prompt.save(os.path.join(kit_dir,'prompts/llama70b-edgar_comparative_qna-query_decomposition_prompt.yaml'))
query_decomposition_prompt = load_prompt(os.path.join(kit_dir,'prompts/llama70b-edgar_comparative_qna-query_decomposition_prompt.yaml'))

In [22]:
# Chain
llm_chain = LLMChain(llm=llm, prompt=query_decomposition_prompt, output_parser=output_parser)

# "lines" is the attribute name of the parsed output
multiquery_retriever = MultiQueryRetriever(
    retriever=vectordb.as_retriever(search_kwargs={
        'k': 3,
        'filter': {'$or': [{'company': {'$eq': 'Uber'}}, {'company': {'$eq': 'Lyft'}}]},
    }), 
    llm_chain=llm_chain, 
    parser_key="lines", 
    verbose = True
)  

question = "What are the revenue breakdowns for the two documents?"

# multiquery results
multiquery_retrieved_docs = multiquery_retriever.get_relevant_documents(
    query=question
)

INFO:langchain.retrievers.multi_query:Generated queries: lines=[]


AssertionError: 

In [19]:
# Define prompt for answering and summarization
summarization_prompt_template = """[INST] <<SYS>>You're a respectful, helpful assistant. Follow these rules:
1. Use only the information provided in the context section.
2. Provide relevant information to answer the question.<<SYS>>
Write an answer to the following question based on the following context:
Question:
{original_question}
Context:
{context}
Answer: [/INST]"""
summarization_prompt = PromptTemplate.from_template(summarization_prompt_template)

In [20]:
summarization_prompt.save(os.path.join(kit_dir,'prompts/llama70b-edgar_comparative_qna-answering_and_summarization_prompt.yaml'))
summarization_prompt = load_prompt(os.path.join(kit_dir,'prompts/llama70b-edgar_comparative_qna-answering_and_summarization_prompt.yaml'))

In [21]:
# Define StuffDocumentsChain
llm_chain = LLMChain(llm=llm, prompt=summarization_prompt)
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

response = stuff_chain.invoke({"input_documents": multiquery_retrieved_docs, 'original_question': question})
print(response['output_text'])

NameError: name 'multiquery_retrieved_docs' is not defined

In [ ]:
questions = [
    "What are the revenue breakdowns for Microsoft and Apple in their respective 10-K reports, and how do they compare in terms of total revenue and revenue from different segments?",
    "What are the key risks mentioned in the risk factors section of both Microsoft and Apple's 10-K reports, and how do they differ in terms of potential impact and mitigation strategies?",
    "How do the corporate governance structures of Microsoft and Apple, as outlined in their 10-K filings, compare in terms of board composition, executive compensation, and shareholder rights?",
    "What are the major investments and acquisitions disclosed in the investment section of Microsoft and Apple's 10-K reports, and how do they reflect each company's strategic priorities and growth strategies?",
    "How do the research and development expenditures disclosed in Microsoft and Apple's 10-K reports compare in terms of absolute spending and percentage of revenue, and what insights can be drawn regarding their innovation efforts?",
    "What are the legal proceedings and regulatory issues disclosed in the legal proceedings section of both Microsoft and Apple's 10-K filings, and how do they differ in terms of nature, severity, and potential impact on the companies?",
    "How do the financial performance metrics such as net income, operating margins, and cash flow ratios disclosed in Microsoft and Apple's 10-K reports compare, and what factors contribute to any observed differences?",
    "What are the geographical revenue breakdowns provided in the geographic segments section of both Microsoft and Apple's 10-K reports, and how do they reflect each company's international presence and market diversification?",
    "How do the sustainability initiatives and environmental disclosures in Microsoft and Apple's 10-K filings compare, including information on energy consumption, carbon footprint, and supply chain sustainability efforts?",
    "What are the forward-looking statements and risk factors outlined in the Management's Discussion and Analysis (MD&A) sections of Microsoft and Apple's 10-K reports, and how do they reflect each company's outlook, challenges, and opportunities in the market?",
]